总体流程：
1. 定位到使用中图分类号搜索的位置；
2. 输入分类号；
3. 点击搜索，获取结果列表；
4. 逐个爬取列表中的 项

In [33]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from loguru import logger
from datetime import datetime
import os
from time import sleep
# from selenium.webdriver.support.select import Select

In [ ]:
logger.add(os.path.join('./logs', 'railway'+str(datetime.now())[:10] + '.log'))

# 一、最原始的方法：直接按照整体流程来写代码

## 1. 搜索给定的中图分类号

In [35]:
# 定义一些常量
INDEX_URL =  'https://s.wanfangdata.com.cn/advanced-search/paper'  # 万方的主页
TIME_OUT = 10  # 最长等待10s
CLS_NUM = 'S969'

In [5]:
# 初始化浏览器
browser = webdriver.Chrome()
wait = WebDriverWait(browser, TIME_OUT)

In [6]:
# 通用爬取函数
def scrape_page(url, condition, locator):
    """通用爬取方法"""
    logger.info(f'scraping {url}')
    try:
        browser.get(url)
        wait.until(condition(locator))
    except TimeoutException:
        logger.error(f'error occurred while scraping {url}')


In [7]:
def scrape_index(cls_num):
    """搜索给定的中图分类号"""
    # 选择查询方式
    scrape_page()

## 主函数

In [39]:
# 1. 打开主页面
browser.get(INDEX_URL)
wait.until(EC.visibility_of_all_elements_located(
    (By.CLASS_NAME, 'advanced-search-box')
))
# 2. 选择查询方式为中图分类号
browser.find_element(by=By.CLASS_NAME, value='ivu-select-selected-value').click()
# select = Select(browser.find_element(by=By.CLASS_NAME, value='ivu-select-dropdown'))
# select.select_by_visible_text("中图分类号")
# browser.find_element(by=By.XPATH, value='//*[contains(text(), "中图分类号")]').click()
lis = browser.find_elements(by=By.CSS_SELECTOR, value='.ivu-select-dropdown-list li')
# wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.ivu-select-dropdown-list li')))
sleep(2)  # 被迫无奈，因此太快这个按钮还没出现
for i in lis:
    if i.text == "中图分类号":
        i.click()
        break

# 3. 查找框中输入中图分类号
input_ = browser.find_element(by=By.CSS_SELECTOR, value='input.ivu-input')
input_.send_keys(CLS_NUM)
button = browser.find_element(by=By.CLASS_NAME, value='submit-btn')
button.click()

In [50]:
# 4. 开始爬取数据（逐个点进去获取数据）
data_list = browser.find_elements(by=By.CSS_SELECTOR, value='.detail-list-wrap > div > div')[:20]
# 5. 尝试点进去第一个爬取数据
data_list[0].find_element(by=By.CLASS_NAME, value='title').click()

In [54]:
# 6.切换到子页面
browser.switch_to.window(browser.window_handles[-1])  # 第一个页面是父页面，第二个是子页面


In [56]:
# 7. 获取相关的数据（暂时省略等待的语句，后续如果需要再补充）
title = browser.find_element(by=By.CLASS_NAME, value='detailTitle').text
summary = browser.find_element(by=By.CSS_SELECTOR, value='.summary > .text-overflow').text


# 成功实现能够爬取数据：2024年3月6日

In [57]:
title, summary

('渔业网衣完整性计算方法研究',
 '针对如何在水产养殖装备设计阶段评估网衣系统完整性,保证渔业生产安全,给出了一种网衣完整性计算方法.在保证群化精度条件下,利用网目群化方法建立数值网衣模型,并将群化模型和AquaSim软件的计算结果进行了对比,验证了该方法的准确性.在此基础上对极端工况下网衣完整性关键影响因素作了分析,包括网目形状、固定方式2种类别型变量,以及网目长度、网线线径、纲绳线径、横纲间距4种连续型变量,对连续型变量作了相关性分析,采用多元回归分析方法拟合得到网衣完整性快速评估模型并进行了验证.结果表明,在结合实际工程情况下对网衣结构设计变量进行试验设计和多元回归分析,实现网衣系统完整性评估是可行的.')

In [49]:
for i in data_list[:20]:
    print(i.find_element(by=By.CLASS_NAME, value='title').text)

渔业网衣完整性计算方法研究
梳草船导航控制系统设计与试验
42 m双渔船并列系泊受力可靠性试验分析
蟹塘梳草船驱动系统设计与试验
单元升降式网箱强度校核
围网网衣清洗装备的设计与打击力试验
近岛礁海域多体浮式渔场平台的动力响应分析
提高犊牛养殖成活率的技术措施
一种管式塑胶水产养殖渔排的设计
噬菌体制剂和杜仲提取物联用减少凡纳滨对虾工厂化养殖前期换水量的可行性探究
光照在甲壳动物养殖中的应用研究进展
江西设施渔业发展状况及前景分析
玻璃钢撑杆浮球式多功能筏式养殖平台与装备的研究(上)
陆基圆池循环水养殖模式调研报告
淡水池塘嵌入式集装槽循环水养殖系统简介
山东省黄河流域渔业产业现状分析及发展建议
北方斑节对虾工厂化养殖成功经验
紫外辐照对海水细菌、浮游植物及理化性质的影响
一种养殖金属网衣变形数值研究
淡水池塘嵌入式集装槽循环水养殖系统2022年养殖效果分析


In [28]:
type(lis[8])

selenium.webdriver.remote.webelement.WebElement

In [26]:
lis[8].click()

In [25]:
lis[9].text
for i in lis:
    print(i.text)

全部
主题
题名或关键词
题名
作者
作者单位
关键词
摘要
中图分类号
DOI
第一作者
期刊-通讯作者
期刊-基金
期刊-刊名
期刊-ISSN/CN
期刊-期
期刊-栏目
学位-专业
学位-学位授予单位
学位-导师
学位-学位
会议名称
会议-主办单位

















































































































































































































































































































































